In [48]:
## Kaggle 수행평가
## Digit Recognizer 문제풀기
# train.csv => 7:3 비율로 나누어서 학습 및 accuracy 확인
# batch 형태로 데이터를 읽어들여서 학습 
# test.csv를 이용하여 prediction결과 도출
# 해당 결과를 이용하여 submission.csv 파일을 생성한 후 Kaggle site에 제출 및 확인. pd.to csv 파일 만드는거 있음

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings(action="ignore")

# tensotflow 그래프 초기화
tf.reset_default_graph()

# Data Loading
df = pd.read_csv("./data/Digit Recognizer/train.csv", sep = "," ,skiprows=0)
# test = pd.read_csv("./data/Digit Recognizer/test.csv", sep = ",")

# display(df.shape)
# display(df.head())

# train 7:3으로 나누기
step_cnt = int(df.shape[0] * 0.7)
# print(step_cnt) # 29399
train_df = df.loc[:step_cnt,:]
test_df  = df.loc[step_cnt:,:]
# display(train_df)

#  x데이터를 0과 1사이의 값으로 변경

#training data set - train
x_data = MinMaxScaler().fit_transform(train_df.loc[:,"pixel0":])
print(type(x_data))
y_data = tf.one_hot(train_df["label"],10)
sess = tf.Session()
y_data = sess.run(y_data)
display(y_data)

#training data set - test
x_data_test = MinMaxScaler().fit_transform(test_df.loc[:,"pixel0":])
y_data_test = tf.one_hot(test_df["label"],10)
y_data_test = sess.run(y_data_test)

display(x_data.shape) # (29400, 784)
display(y_data.shape) # (29400, 10)

# placeholder
X = tf.placeholder(shape=[None, 784], dtype=tf.float32) 
Y = tf.placeholder(shape=[None, 10], dtype=tf.float32)
keep_prob = tf.placeholder(dtype=tf.float32)

# Weight & bias
# W1 = tf.Variable(tf.random_normal([784,256]), name="weight1")
W1 = tf.get_variable("weight1", shape=[784,256], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]), name="bias1")
# layer1 = tf.nn.relu(tf.matmul(X,W1) + b1) 
_layer1 = tf.nn.relu(tf.matmul(X,W1) + b1) 
layer1 = tf.nn.dropout(_layer1, keep_prob = keep_prob)

# W2 = tf.Variable(tf.random_normal([256,256]), name="weight2")
W2 =  tf.get_variable("weight2", shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]), name="bias2")
# layer2 = tf.nn.relu(tf.matmul(layer1,W2) + b2)
_layer2 = tf.nn.relu(tf.matmul(layer1,W2) + b2)
layer2 = tf.nn.dropout(_layer2, keep_prob = keep_prob)

# W3 = tf.Variable(tf.random_normal([256,10]), name="weight3")
W3 = tf.get_variable("weight3", shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]), name="bias3")

# Hypothesis
logits = tf.matmul(layer2,W3) + b3  
H = tf.nn.relu(logits)

# Cost Function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

# train node 생성
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# epoch 학습진행 , 데이터 슬라이싱
training_epoch = 100
batch_size = 100 

for step in range(training_epoch): 
    num_of_iter = int( x_data.shape[0] /  batch_size) 
    cost_val = 0
    # (x데이터,y데이터)
    start_idx = 0
    end_idx = 0
    for i in range(num_of_iter):
        # batch_x, batch_y = mnist.train.next_batch(batch_size) 
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        
        batch_x = x_data[start_idx:end_idx,:]
        batch_y = y_data[start_idx:end_idx,:]
        
        _, cost_val = sess.run([train,cost], feed_dict={X:batch_x , Y:batch_y, keep_prob:1.0})
        
    if step % 10 == 0:
        print(cost_val)     

# Accuracy(정확도) 측정
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

result = sess.run([accuracy], feed_dict={X:x_data, Y:y_data, keep_prob:1.0})

print("정확도 :{}".format(result))

#predict check
df_test = pd.read_csv("./data/Digit Recognizer/test.csv",sep=",",skiprows=0)
x_data_test = MinMaxScaler().fit_transform(df_test)

predict = tf.argmax(H,1)
result = sess.run(predict, feed_dict={X:x_data_test, keep_prob: 1.0})
df = pd.DataFrame({
    "ImageId": [i for i in range(1,28001)],
    "Label": result
})
#display(result)
df.to_csv("./data/Digit Recognizer/submission.csv", index=False)

<class 'numpy.ndarray'>


array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

(29400, 784)

(29400, 10)

0.20250611
0.004919688
0.0032596055
0.0011879979
1.9788858e-05
5.275711e-06
1.0633271e-06
2.0503909e-07
3.6954845e-08
7.152556e-09
정확도 :[1.0]


In [53]:
## Digit Recognizer 문제풀기 + CNN적용
# train.csv => 7:3 비율로 나누어서 학습 및 accuracy 확인
# batch 형태로 데이터를 읽어들여서 학습 
# test.csv를 이용하여 prediction결과 도출
# 해당 결과를 이용하여 submission.csv 파일을 생성한 후 Kaggle site에 제출 및 확인. pd.to csv 파일 만드는거 있음

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings(action="ignore")

# tensotflow 그래프 초기화
tf.reset_default_graph()

# Data Loading
df = pd.read_csv("./data/Digit Recognizer/train.csv", sep = "," ,skiprows=0)
# test = pd.read_csv("./data/Digit Recognizer/test.csv", sep = ",")

# display(df.shape)
# display(df.head())

# train 7:3으로 나누기
step_cnt = int(df.shape[0] * 0.7)
# print(step_cnt) # 29399
train_df = df.loc[:step_cnt,:]
test_df  = df.loc[step_cnt:,:]
# display(train_df)

#  x데이터를 0과 1사이의 값으로 변경

#training data set - train
x_data = MinMaxScaler().fit_transform(train_df.loc[:,"pixel0":])
print(type(x_data))
y_data = tf.one_hot(train_df["label"],10)
sess = tf.Session()
y_data = sess.run(y_data)
display(y_data)

#training data set - test
x_data_test = MinMaxScaler().fit_transform(test_df.loc[:,"pixel0":])
y_data_test = tf.one_hot(test_df["label"],10)
y_data_test = sess.run(y_data_test)

display(x_data.shape) # (29400, 784)
display(y_data.shape) # (29400, 10)

# placeholder
X = tf.placeholder(shape=[None, 784], dtype=tf.float32) 
Y = tf.placeholder(shape=[None, 10], dtype=tf.float32)
keep_prob = tf.placeholder(dtype=tf.float32)

# Convolution 
X_img = tf.reshape(X,[-1,28,28,1])

## Convolution Layer1 
L1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu) # 3x3 이미지 32개 # kernel_size : 필터의 크기
L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2], padding="SAME", strides=2)

## Convolution Layer2
L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu) # 3x3 이미지 64개 # kernel_size : 필터의 크기
L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2], padding="SAME", strides=2)
# karas.layers.max_pooling2d 
print(L2.shape) 

L2 = tf.reshape(L2,[-1,7*7*64])

# Weight & bias
# W1 = tf.Variable(tf.random_normal([784,256]), name="weight1")
W1 = tf.get_variable("weight1", shape=[7*7*64,256], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]), name="bias1")
# layer1 = tf.nn.relu(tf.matmul(X,W1) + b1) 
_layer1 = tf.nn.relu(tf.matmul(L2,W1) +b1)
layer1 = tf.nn.dropout(_layer1, keep_prob = keep_prob)

# W2 = tf.Variable(tf.random_normal([256,256]), name="weight2")
W2 =  tf.get_variable("weight2", shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]), name="bias2")
# layer2 = tf.nn.relu(tf.matmul(layer1,W2) + b2)
_layer2 = tf.nn.relu(tf.matmul(layer1,W2) + b2)
layer2 = tf.nn.dropout(_layer2, keep_prob = keep_prob)

# W3 = tf.Variable(tf.random_normal([256,10]), name="weight3")
W3 = tf.get_variable("weight3", shape=[256,10], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]), name="bias3")

# Hypothesis
logits = tf.matmul(layer2,W3) + b3  
H = tf.nn.relu(logits)

# Cost Function
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

# train node 생성
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
# optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
# trian = optimizer.minimize(cost)

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# epoch 학습진행 , 데이터 슬라이싱
training_epoch = 100
batch_size = 100 

for step in range(training_epoch): 
    num_of_iter = int( x_data.shape[0] /  batch_size) 
    cost_val = 0
    # (x데이터,y데이터)
    start_idx = 0
    end_idx = 0
    for i in range(num_of_iter):
        # batch_x, batch_y = mnist.train.next_batch(batch_size) 
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
#         if end_idx > :
#             end_idx = batch_size
        
        batch_x = x_data[start_idx:end_idx,:]
        batch_y = y_data[start_idx:end_idx,:]
        
        _, cost_val = sess.run([train,cost], feed_dict={X:batch_x , Y:batch_y, keep_prob:1.0}) # 0.5-0.7로
        
    if step % 10 == 0:
        print(cost_val)     

# Accuracy(정확도) 측정
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

result = sess.run([accuracy], feed_dict={X:x_data, Y:y_data, keep_prob:1.0})

print("정확도 :{}".format(result))

#predict check
df_test = pd.read_csv("./data/Digit Recognizer/test.csv",sep=",",skiprows=0)
x_data_test = MinMaxScaler().fit_transform(df_test)

predict = tf.argmax(H,1)
result = sess.run(predict, feed_dict={X:x_data_test, keep_prob: 1.0})
df = pd.DataFrame({
    "ImageId": [i for i in range(1,28001)],
    "Label": result
})
#display(result)
df.to_csv("./data/Digit Recognizer/submission.csv", index=False)

<class 'numpy.ndarray'>


array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

(29400, 784)

(29400, 10)

(?, 7, 7, 64)
0.09422663
0.0014924891
0.0020208089
1.3721731e-05
3.683524e-07
3.814692e-08
3.5762782e-09
0.0
0.0
0.0
정확도 :[1.0]


In [52]:
## Titanic: Machine Learning from Disaster 문제 풀기
# train.csv => 7:3 비율로 나누어서 학습 및 accuracy 확인
# batch 형태로 데이터를 읽어들여서 학습 
# test.csv를 이용하여 prediction결과 도출
# 해당 결과를 이용하여 gender_submission.csv 파일을 생성한 후 Kaggle site에 제출 및 확인. pd.to csv 파일 만드는거 있음

#PassengerId 탑승자id,Survived 구조여부 (1 구조 0 구조못함)
#Pclass 좌석등급, Name 이름, Sex 성별, Age 나이,SibSp 형제수,Parch 부모/자식수,
#Ticket 티켓번호,Fare 요금, Cabin 객실, Embarked 탑승한 곳

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import warnings

warnings.filterwarnings(action="ignore")

# tensotflow 그래프 초기화
tf.reset_default_graph()

# Data Loading
df = pd.read_csv("./data/Titanic_/train.csv", sep=",", skiprows=0)
# display(df)

# 데이터 정제
df2 = df[["Survived","Pclass","Sex","Age","Parch","Fare"]]
df2.dropna(how="any", inplace=True)

df2["Sex"] = df2['Sex'].apply(lambda x: '1' if x == 'male' else '2')

# 데이터 정제
# df.drop(["Name","Ticket","Cabin"], axis = 1, inplace = True )
# df.dropna(how = "any", inplace = True) # NaN이 있는 모든 row 삭제
# df.loc[df["Sex"] == "male","Sex"] = 1 # 남자는 1 여자는 2
# df.loc[df["Sex"] == "female","Sex"] = 2
# df.loc[:,"Age"] = (df["Age"]//10) # 연령을 처리하기 쉽게 하기위해 나누기10 / 0-9세는 0, 10-19세는 1, 20-29는 2 ... 표현
# df.loc[df["Embarked"] == "S","Embarked"] = 1  # Embarked == 1: Southampton
# df.loc[df["Embarked"] == "C","Embarked"] = 2  # Embarked == 2: Cherbourg
# df.loc[df["Embarked"] == "Q","Embarked"] = 3  # Embarked == 3: Queenstown
# df["Fare"] = pd.qcut(df["Fare"],5,labels=[1,2,3,4,5]) # 전체 요금을 5개로 나눠줌
# display(df)

# 7:3으로 나누기
step_cnt = int(df2.shape[0] * 0.7)
# print(step_cnt) # 623
train_df = df2.loc[:step_cnt,:]
test_df  = df2.loc[step_cnt:,:]
# bdisplay(train_df)

# train
df_x = train_df[["Pclass","Sex","Age","Parch","Fare"]]
df_y = train_df[["Survived"]]
print(type(df_x))
# display(df_x)

# test
df_x_test = test_df[["Pclass","Sex","Age","Parch","Fare"]]
df_y_test = test_df[["Survived"]]

#training data set - train
x_data = MinMaxScaler().fit_transform(df_x)
y_data = df_y.values

display(x_data.shape) # (398, 5)
display(y_data.shape) # (398, 1)

#training data set - test
x_data_test = MinMaxScaler().fit_transform(df_x_test.values)
y_data_test = df_y_test

print("x_data_test shape: {}".format(x_data_test.shape)) # (317, 5)
print("y_data_test shape: {}".format(y_data_test.shape)) # (317, 1)
print(type(x_data_test))
print(type(y_data_test))

# placeholder
X = tf.placeholder(shape=[None, 5], dtype=tf.float32) 
Y = tf.placeholder(shape=[None, 1], dtype=tf.float32)
keep_prob = tf.placeholder(dtype=tf.float32)

# Weight & bias
W1 = tf.get_variable("weight1", shape=[5,20], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([20]), name="bias1")
# layer1 = tf.nn.relu(tf.matmul(X,W1) + b1) 
_layer1 = tf.nn.relu(tf.matmul(X,W1) + b1) 
layer1 = tf.nn.dropout(_layer1, keep_prob = keep_prob)

W2 =  tf.get_variable("weight2", shape=[20,20], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([20]), name="bias2")
# layer2 = tf.nn.relu(tf.matmul(layer1,W2) + b2)
_layer2 = tf.nn.relu(tf.matmul(layer1,W2) + b2)
layer2 = tf.nn.dropout(_layer2, keep_prob = keep_prob)


W3 = tf.get_variable("weight3", shape=[20,1], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([1]), name="bias3")

# Hypothesis
logits = tf.matmul(layer2,W3) + b3  
H = tf.nn.relu(logits)

# Cost Function
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y))

# train node 생성
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

# Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# epoch 학습진행 , 데이터 슬라이싱
training_epoch = 100
batch_size = 100 

for step in range(training_epoch): 
    num_of_iter = int( x_data.shape[0] /  batch_size) 
    cost_val = 0
    # (x데이터,y데이터)
    start_idx = 0
    end_idx = 0
    for i in range(num_of_iter):
        # batch_x, batch_y = mnist.train.next_batch(batch_size) 
        start_idx = i * batch_size
        end_idx = start_idx + batch_size
        
        batch_x = x_data[start_idx:end_idx,:]
        batch_y = y_data[start_idx:end_idx,:]
        
        _, cost_val = sess.run([train,cost], feed_dict={X:batch_x , Y:batch_y, keep_prob:1.0})
        
    if step % 10 == 0:
        print(cost_val)     


# Accuracy 정확도
predict = tf.argmax(H,1)
correct = tf.equal(predict, tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

# print(type(df_x_test))

result = sess.run([accuracy], feed_dict={X:x_data_test, Y:y_data_test, keep_prob:1.0})

print("정확도 :{}".format(result))

##################################################################################

#predict check
df_t = pd.read_csv("./data/Titanic_/test.csv",sep=",",skiprows=0)

df_t2 = df_t[["PassengerId","Pclass","Sex","Age","Parch","Fare"]]

df_t2["Sex"] = df_t2['Sex'].apply(lambda x: '1' if x == 'male' else '2')

df_ids = df_t2[["PassengerId"]]
df_test = df_t2[["Pclass","Sex","Age","Parch","Fare"]]
#print(df_t2)
test_x_data = MinMaxScaler().fit_transform(df_test.values)
#print(test_x_data)

# predict = tf.argmax(H,1)
predict = tf.cast(H >= 0.5, dtype=tf.float32)
result = sess.run(predict, feed_dict={X:test_x_data, keep_prob: 1.0})
print(result)
df = pd.DataFrame({
    "PassengerId": [ str(i[0]) for i in df_ids.values ],
    "Survived": [ int(i[0]) for i in result ]
})
# print(df)
#display(result)
df.to_csv("./data/Titanic_/gender_submission.csv", index=False)

<class 'pandas.core.frame.DataFrame'>


(398, 5)

(398, 1)

x_data_test shape: (317, 5)
y_data_test shape: (317, 1)
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
0.6903784
0.6799202
0.6401634
0.6062219
0.56409854
0.52428293
0.4993717
0.48544303
0.47652692
0.47096008
정확도 :[1.0]
[[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.